In [ ]:
from crewai import Agent, Task, Crew, LLM
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize LLM
llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.3
)

# =============================================================================
# TRAVEL PLANNING AGENTS
# =============================================================================

# 1. WEATHER RESEARCH AGENT
weather_agent = Agent(
    role="Weather Research Specialist",
    goal="Analyze weather conditions at destinations to optimize travel timing and activity planning",
    backstory="""You are a meteorology expert who specializes in travel weather planning.
    You understand how weather impacts different activities and can recommend the best times
    to visit destinations based on seasonal patterns, rainfall, temperature, and other factors.""",
    verbose=False,
    llm=llm,
    max_iter=1,
    allow_delegation=False
)

# 2. BUDGET ANALYST AGENT
budget_agent = Agent(
    role="Family Travel Budget Specialist",
    goal="Optimize travel budgets for families and groups by finding the best deals and cost-effective options",
    backstory="""You are a family travel financial expert who specializes in group bookings and
    family-friendly cost optimization. You understand group discounts, family rates, child pricing,
    and how to maximize value when traveling with multiple people including children.""",
    verbose=False,
    llm=llm,
    max_iter=1,
    allow_delegation=False
)

# 3. ACCOMMODATION SPECIALIST AGENT
accommodation_agent = Agent(
    role="Family & Group Accommodation Expert",
    goal="Find optimal lodging for groups and families, including multi-room setups and child-friendly amenities",
    backstory="""You are a family travel accommodation specialist with expertise in group bookings.
    You understand family needs, child safety requirements, space requirements for groups,
    and can find properties that work well for multiple adults and children together. For eg airbnb to fit whole group as well as cooking amenities etc""",
    verbose=False,
    llm=llm,
    max_iter=1,
    allow_delegation=False
)

# 4. ACTIVITIES & ATTRACTIONS AGENT
activities_agent = Agent(
    role="Family & Group Activities Specialist",
    goal="Curate family-friendly activities suitable for mixed age groups including children",
    backstory="""You are a family travel expert specializing in activities that work for groups
    with different ages and interests. You know which attractions are suitable for children,
    offer group discounts, and can balance adult interests with kid-friendly options.""",
    verbose=False,
    llm=llm,
    max_iter=1,
    allow_delegation=False
)

# 5. ITINERARY COORDINATOR AGENT
coordinator_agent = Agent(
    role="Master Itinerary Coordinator",
    goal="Synthesize all agent inputs into a cohesive, practical, and optimized travel itinerary",
    backstory="""You are a master travel planner with years of experience coordinating complex trips.
    You excel at taking disparate information and creating seamless, logical itineraries that
    maximize traveler satisfaction while staying within constraints.""",
    verbose=False,
    llm=llm,
    max_iter=1,
    allow_delegation=False
)

# =============================================================================
# TRAVEL PLANNING TASKS
# =============================================================================

def create_travel_plan(destination, travel_dates, budget, departure_city, traveler_preferences, group_size="1 adult", kids_info=""):
    """
    Create a comprehensive travel plan using multiple specialized agents

    Args:
        destination (str): Where to travel
        travel_dates (str): When to travel (e.g., "March 15-22, 2024")
        budget (str): Total budget (e.g., "$3000")
        departure_city (str): Where traveling from
        traveler_preferences (str): Interests, activity level, etc.
        group_size (str): Number of travelers (e.g., "6 adults, 2 children")
        kids_info (str): Ages and needs of children
    """

    # TASK 1: Weather Analysis
    weather_task = Task(
        description=f"""Analyze the weather conditions for {destination} during {travel_dates}.

        Provide:
        - Daily weather forecast and seasonal patterns
        - Best times of day for outdoor activities
        - What to pack for weather conditions
        - Any weather-related travel considerations
        - Recommendations for weather-appropriate activities

        Destination: {destination}
        Travel Dates: {travel_dates}""",
        agent=weather_agent,
        expected_output="Detailed weather analysis with activity recommendations and packing suggestions."
    )

    # TASK 2: Budget Analysis
    budget_task = Task(
        description=f"""Analyze travel costs for a group trip and optimize the budget.

        Research and provide:
        - Group flight costs from {departure_city} to {destination} during {travel_dates}
        - Group/family discounts and deals available
        - Budget breakdown for group travel (flights, accommodation, food, activities)
        - Cost per person vs total group cost analysis
        - Child pricing considerations where applicable
        - Money-saving strategies for group travel

        Group Details: {group_size}
        Children Information: {kids_info}
        Total Budget: {budget}
        Departure City: {departure_city}
        Destination: {destination}
        Travel Dates: {travel_dates}""",
        agent=budget_agent,
        expected_output="Group travel budget analysis with per-person and total costs, including child considerations."
    )

    # TASK 3: Accommodation Research
    accommodation_task = Task(
        description=f"""Find optimal accommodation for group travel with children.

        Requirements for group accommodation:
        - Space for {group_size}
        - Child-friendly amenities and safety features
        - Multiple bedrooms/bathrooms as needed
        - Kitchen facilities for group meals (cost savings)
        - Location convenient for planned activities
        - Group booking discounts where possible

        Consider:
        - Budget constraints from budget analysis
        - Child ages: {kids_info}
        - Safety and convenience for families
        - Airbnb, vacation rentals, or family hotel options

        Budget: {budget}
        Group: {group_size}
        Destination: {destination}
        Travel Dates: {travel_dates}
        Children: {kids_info}

        Use insights from previous weather and budget analyses.""",
        agent=accommodation_agent,
        expected_output="Top 3-5 family/group accommodation options with detailed analysis of suitability for children.",
        context=[weather_task, budget_task]
    )

    # TASK 4: Activities Planning
    activities_task = Task(
        description=f"""Plan family-friendly activities suitable for mixed age groups.

        Requirements:
        - Activities suitable for children ages: {kids_info}
        - Group-friendly attractions with capacity for {group_size}
        - Mix of adult interests and child-friendly options
        - Consider attention spans and energy levels of children
        - Group discounts and family rates
        - Weather-appropriate indoor and outdoor options
        - Safety considerations for children

        Plan based on:
        - Weather conditions and timing
        - Group budget allocated for activities
        - Adult preferences: {traveler_preferences}
        - Child-friendly logistics and facilities

        Group Details: {group_size}
        Children: {kids_info}
        Destination: {destination}
        Travel Dates: {travel_dates}

        Incorporate weather, budget, and accommodation insights.""",
        agent=activities_agent,
        expected_output="Family-friendly day-by-day activity plan with child considerations, costs, and group logistics.",
        context=[weather_task, budget_task, accommodation_task]
    )

    # TASK 5: Final Itinerary Coordination
    final_itinerary_task = Task(
        description=f"""Create a comprehensive family/group travel itinerary.

        Synthesize all analyses to create:
        - Day-by-day detailed itinerary suitable for {group_size}
        - Complete budget breakdown (total and per-person costs)
        - Group booking timeline and coordination checklist
        - Packing lists for adults and children based on weather
        - Transportation logistics for group travel
        - Child safety considerations and emergency contacts
        - Backup indoor plans for weather or child meltdowns
        - Meal planning for group dining
        - Nap time and rest considerations for children

        Ensure the itinerary addresses:
        - Group coordination and logistics
        - Child-friendly timing and pacing
        - Budget efficiency for group travel
        - Weather-appropriate planning
        - Safety and convenience for families

        Trip Details:
        - Group: {group_size}
        - Children: {kids_info}
        - Destination: {destination}
        - Travel Dates: {travel_dates}
        - Budget: {budget}
        - Departure: {departure_city}
        - Adult Preferences: {traveler_preferences}""",
        agent=coordinator_agent,
        expected_output="Complete family/group travel itinerary with child considerations, group logistics, and detailed practical planning.",
        context=[weather_task, budget_task, accommodation_task, activities_task]
    )

    # Create and execute the crew
    travel_crew = Crew(
        agents=[weather_agent, budget_agent, accommodation_agent, activities_agent, coordinator_agent],
        tasks=[weather_task, budget_task, accommodation_task, activities_task, final_itinerary_task],
        verbose=False,
        process="sequential"  # Tasks execute in order, passing context
    )

    return travel_crew.kickoff()

# =============================================================================
# EXAMPLE USAGE
# =============================================================================

if __name__ == "__main__":
    # Example: Queenstown family trip
    destination = "Queenstown, New Zealand"
    travel_dates = "October 15-22, 2025"
    budget = "$10,000 AUD"
    departure_city = "Sydney, Australia"
    group_size = "6 adults, 2 children"
    kids_info = "Ages 3 and 10 - need child-friendly activities, shorter attention spans"
    preferences = """
    - Mix of adventure activities (for adults) and family-friendly attractions
    - Scenic photography opportunities
    - Indian food experiences suitable for children
    - Balance of activity and relaxation for mixed group
    - Preference for Airbnb with cooking, laundry amenities or vacation rental for group space close to city centre
    - Some adults interested in wine/adventure, others prefer sightseeing
    - Need child-safe activities and amenities
    """

    print("👨‍👩‍👧‍👦 FAMILY & GROUP TRAVEL PLANNER")
    print("=" * 50)
    print(f"Planning trip to: {destination}")
    print(f"Group: {group_size}")
    print(f"Children: {kids_info}")
    print(f"Travel dates: {travel_dates}")
    print(f"Budget: {budget}")
    print(f"From: {departure_city}")
    print("=" * 50)
    print("\n🤖 Specialized family travel agents working...\n")

    # Execute the family travel planning
    result = create_travel_plan(
        destination=destination,
        travel_dates=travel_dates,
        budget=budget,
        departure_city=departure_city,
        traveler_preferences=preferences,
        group_size=group_size,
        kids_info=kids_info
    )

    print("\n" + "=" * 50)
    print("📋 YOUR FAMILY GROUP TRAVEL ITINERARY")
    print("=" * 50)
    print(result)

# =============================================================================
# QUICK TEST FUNCTION
# =============================================================================

def family_queenstown_trip():
    """Plan the specific Queenstown family trip"""
    print("🏔️👨‍👩‍👧‍👦 QUEENSTOWN FAMILY ADVENTURE")
    print("-" * 40)

    result = create_travel_plan(
        destination="Queenstown, New Zealand",
        travel_dates="October 15-22, 2025",
        budget="$10,000 AUD",
        departure_city="Sydney, Australia",
        traveler_preferences="""
        Adults enjoy: adventure activities, wine tasting, scenic views, photography
        Need: child-friendly alternatives, family dining options, flexible schedule
        Prefer: Airbnb with kitchen, central location, safe for children
        Balance: adult adventure time with family-friendly activities
        """,
        group_size="6 adults, 2 children",
        kids_info="3-year-old (needs naps, simple activities) and 10-year-old (more adventurous, longer attention span)"
    )

    print(result)

# Uncomment to run the Queenstown family trip
# family_queenstown_trip()